In [4]:
import sys
sys.path.append('../Auxiliary/')
from helpsters import *
# import packages 

import rasterio as rio
from rasterio.transform import from_origin

In [6]:
files = sorted(getFilelist('/data/Aldhani/eoagritwin/et/Sentinel3/raw', '.nc'))
year = 2024
# get a subset of files for that year
yearFiles = [file for file in files if int(file.split('/')[-1].split('_')[-1][0:4]) == year]

# make dictionary that stores .tif names and related accDates to easier search to close observations with S2 data
keys = ['filename', 'accDateTimes']
vals = [list() for i in range(len(keys))]
lookUp = dict(zip(keys, vals))


In [7]:
# set storPath for exported tiffs
storPath = '/data/Aldhani/eoagritwin/et/Sentinel3/tiffs/lst/'
cont = []
# loop over files and export to .tif at location storPath
for file in yearFiles:

    accDateTimes = getAllDatesS3(file) # possible to take annual subset if entire files list would be passed here
    convertNCtoTIF(file, storPath, file.split('/')[-1].split('.')[0] + '.tif', accDateTimes, False, True)

    # # write filenames and corresponding dates into dictionary for easier datesearch later
    # lookUp['filename'].append(file.split('.nc')[0])
    # lookUp['accDateTimes'].append(accDateTimes)


    # cont = []
   
    # #for file in yearFiles:
    # aa  = getDataFromNC(file)
    # accDateTimes = getAllDatesS3(file)
    # df = pd.Series(accDateTimes)
    # counts_per_day = df.dt.floor("D").value_counts().sort_index()
    # t2 = np.cumsum(counts_per_day)
    # t1 = np.insert(t2, 0 ,0)

    # for l in range(len(counts_per_day)):
    #     # cont.append(np.any(~np.isnan(aa[:, :, t1[l]:t2[l]]),axis=2)) minimum dail obs
    #     cont.append(np.nansum(~np.isnan(aa[:, :, t1[l]:t2[l]]),axis=2))

    # exportNCarrayDerivatesInt(file, storPath, f'Minimum_DailyObservations_{('_').join(file.split('_')[-1].split('-')[:2])}.tif', 'monthly_sum_of_daily_obs', np.nansum(np.dstack((cont)), axis = 2), True)
#xportNCarrayDerivatesInt(file, storPath, f'Minimum_DailyObservations_{file.split('_')[-1].split('-')[0]}.tif', 'annual_sum_of_daily_obs', np.nansum(np.dstack((cont)), axis = 2), True)
#exportNCarrayDerivatesInt(file, storPath, f'DailyObservations_{file.split('_')[-1].split('-')[0]}.tif', 'annual_sum_of_all_daily_obs', np.nansum(np.dstack((cont)), axis = 2), True)

In [ ]:
dat = np.asarray((np.nan))
dat.astype(np.uint16)

In [ ]:
ncfile = xr.open_dataset(yearFiles[0])
arr = ncfile[[b for b in ','.join(ncfile.data_vars.keys()).split(',') if b == 'confidence_in'][0]].to_numpy() 
np.unique(arr)
